In [0]:
from pyspark.sql import SparkSession
import pandas as pd
import random

# Initialize Spark Session
spark = SparkSession.builder.appName("Databricks_Catalog_Read").getOrCreate()

# Define the catalog and schema
catalog_name = "workspace"
schema_name = "default"

# Read data from Databricks tables and convert them to Pandas DataFrames

table_full_path = f"{catalog_name}.{schema_name}.{'geo'}"
    
# Read the table using Spark
df_spark = spark.read.table(table_full_path)
    
# Convert to Pandas DataFrame
globals()['df_geo'] = df_spark.toPandas()

In [0]:
# Show all rows
pd.set_option('display.max_rows', None)

In [0]:
df_geo

In [0]:
import pandas as pd

# Create 'coordinates' column only if 'latitude' and 'longitude' exist
if 'latitude' in df_geo.columns and 'longitude' in df_geo.columns:
    df_geo['coordinates'] = df_geo[['latitude', 'longitude']].apply(lambda x: [x[0], x[1]], axis=1)
    
    # Drop 'latitude' and 'longitude' only if they exist
    df_geo.drop(columns=['latitude', 'longitude'], inplace=True)

# Convert 'timestamp' column to datetime format
df_geo['timestamp'] = pd.to_datetime(df_geo['timestamp'])

# Reorder the columns
df_geo = df_geo[['ind', 'country', 'coordinates', 'timestamp']]

df_geo

In [0]:
# Convert Pandas DataFrame to Spark DataFrame
df_geo_spark = spark.createDataFrame(df_geo)

#Save as managed delta table
df_geo_spark.write.mode("overwrite").saveAsTable("workspace.default.df_geo")

